In [7]:
import pygame
import math

pygame.init()

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

WIDTH, HEIGHT = 1000, 800
my_font = pygame.font.SysFont('dejavuserif', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong")

paddle_img = pygame.image.load('paddle.jpg')
bg = pygame.image.load("bg.jpg")
bg = pygame.transform.scale(bg, (WIDTH, HEIGHT))

class Paddle(pygame.sprite.Sprite):
    def __init__(self, centerx, centery):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.transform.scale(paddle_img, (40, 250))
        self.rect = self.image.get_rect()
        self.rect.centerx = centerx
        self.rect.centery = centery
        self.click = False

class Ball:
    def __init__(self, posx, posy, radius, speed, color):
        self.posx = posx
        self.posy = posy
        self.radius = radius
        self.speed = speed
        self.color = color
        self.xFac = 1
        self.yFac = -1
        self.ball = pygame.draw.circle(screen, self.color, (self.posx, self.posy), self.radius)
        self.firstTime = 1
 
    def display(self):
        self.ball = pygame.draw.circle(
            screen, self.color, (self.posx, self.posy), self.radius)
 
    def update(self):
        self.posx += self.speed*self.xFac
        self.posy += self.speed*self.yFac
 
        # If the ball hits the top or bottom surfaces, 
        # then the sign of yFac is changed and 
        # it results in a reflection
        if self.posy <= 0 or self.posy >= HEIGHT:
            self.yFac *= -1
 
        if self.posx <= 0 and self.firstTime:
            self.firstTime = 0
            return 1
        elif self.posx >= WIDTH and self.firstTime:
            self.firstTime = 0
            return -1
        else:
            return 0
 
    def reset(self):
        self.posx = WIDTH//2
        self.posy = HEIGHT//2
        self.xFac *= -1
        self.firstTime = 1
 
    # Used to reflect the ball along the X-axis
    def hit(self):
        self.xFac *= -1
 
    def getRect(self):
        return self.ball

def main():
    running = True

    paddles = pygame.sprite.Group()
    
    paddle_l = Paddle(30, HEIGHT / 2)
    paddle_r = Paddle(WIDTH - 30, HEIGHT / 2)

    paddles.add(paddle_l)
    paddles.add(paddle_r)
    
    ball = Ball(WIDTH // 2, HEIGHT // 2, 10, 4, WHITE)
    
    while running:
        screen.fill(BLACK)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if paddle_l.rect.collidepoint(pos):
                    paddle_l.click = True
                if paddle_r.rect.collidepoint(pos):
                    paddle_r.click = True
            if event.type == pygame.MOUSEBUTTONUP:
                paddle_l.click = False
                paddle_r.click = False
            
            if paddle_l.click:
                current_pos = pygame.mouse.get_pos()
                if paddle_l.rect.bottomleft[1] <= HEIGHT and paddle_l.rect.topleft[1] >= 0:
                    paddle_l.rect.centery = current_pos[1]
                    
                # fix overshoot    
                if paddle_l.rect.topleft[1] < 0:
                    paddle_l.rect.centery = paddle_l.rect.height/2
                if paddle_l.rect.bottomleft[1] > HEIGHT:
                    paddle_l.rect.centery = HEIGHT - paddle_l.rect.height/2

            if paddle_r.click:
                current_pos = pygame.mouse.get_pos()
                if paddle_r.rect.bottomleft[1] <= HEIGHT and paddle_r.rect.topleft[1] >= 0:
                    paddle_r.rect.centery = current_pos[1]
                    
                # fix overshoot    
                if paddle_r.rect.topleft[1] < 0:
                    paddle_r.rect.centery = paddle_r.rect.height/2
                if paddle_r.rect.bottomleft[1] > HEIGHT:
                    paddle_r.rect.centery = HEIGHT - paddle_r.rect.height/2


        if pygame.Rect.colliderect(ball.getRect(), paddle_l.rect):
            ball.hit()

        if pygame.Rect.colliderect(ball.getRect(), paddle_r.rect):
            ball.hit()

        ball.update()

        screen.blit(bg, (0, 0))
        paddles.draw(screen)
        ball.display()
        pygame.display.update()


if __name__ == "__main__":
    main()
    pygame.quit()